# Historical Quasar Observations
***
## Learning Goals
By the end of this tutorial, you will:
* Understand how to query data on a target from the MAST archive.
* Be able to plot a Historial Observational Coverage plot of a target.
* Know how to plot a Hubble Spectrum for a target.

## Introduction
Quasars are extremely luminous astronomical objects that can be found at the center of some galaxies. They are powered by gas spiraling at high velocity into a super-massive black hole. The brightest quasars are capable of outshining all the stars in their galaxy; they can be seen from billions of light-years away. The first quasar ever discovered is called **3c273**. It is one of the most luminous quasars and therefore one of the most luminous objects in the observable universe. It is at a distance of 749 Megaparsecs [1 Megaparsec = 1 million parsecs = 3.26 million lightyears] with an absolute magnitude of −26.7, meaning that if it were at a distance of 10 parsecs, it would be as bright in our sky as the Sun. 

Quasar **3c273** is the our target in this tutorial. We will first search the MAST Archive for all the observations of this quasar. Then, we will display those observations in a historic coverage plot; that is, a plot of the wavelengths in which **3c273** was observed for a given year. This will help us understand what the history of observations of this quasar looks over time. Finally, we will plot a spectrum from one of the observations. 

### Workflow

* [Imports](#Imports)
* [Historic Observation Coverage](#Historical-Observation-Coverage)
    * [Query the MAST Archive](#Query-the-MAST-Archive-by-Object-Name)
    * [Create Plotting Variables](#Create-Plotting-Variables)
    * [Plotting Historical Observation Coverage](#Plotting-Historical-Observation-Coverage)
* [Plot a Spectrum](#Plot-a-Spectrum)
    * [Query HST for Spectra](#Query-HST-for-Spectra)
    * [List Available Instrument and Filter Combinations](#List-Available-Instrument-and-Filter-Combinations)
    * [Select Desired Observations](#Select-Observations-with-a-Specific-Instrument-and-Filter-Combination)
    * [Filter for Relevant Products](#Filtering-for-Relevant-Products)
    * [Download the Data](#Download-the-Data-to-Plot)
    * [Plot the Spectrum](#Plot-the-Spectrum)
* [Exercises](#Exercises)

## Imports
The following cell holds the imported packages. These packages are necessary for running the rest of the cells in this notebook. A description of each import is as follows:

* `numpy` to handle array functions
* `pandas` to handle date conversions
* `fits` from astropy.io for accessing FITS files
* `Table` from astropy.table for creating tidy tables of the data
* `matplotlib.pyplot` for plotting data
* `Mast` and `Observations` from astroquery.mast for querying data and observations from the MAST archive

In [ ]:
from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Mast, Observations

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Historical Observation Coverage
The MAST Archive has data from as early as the 1970s. In this section, we'll search for a target by name and examine its observational history. We'll create a plot of this history, showing when the target was observed, in what wavelength, and by which mission.

## Query the MAST Archive by Object Name

We are going to use the **astroquery.mast** `Observations` package to gather our data from the MAST Archive. In this tutorial, we will use the `Observations.query_object()` function which takes the name of the target and an optional radius. If you don't specify a radius, 0.2 degrees will be used by default. For more information about queries you can read the [astroquery.mast readthedocs](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

In [ ]:
# Define target name
target_name = "3c273"

# We'll name the data "obs_table" for "observations table"
obs_table = Observations.query_object(target_name)

In [ ]:
# Print out the first 10 entries with limited columns, if you want to see a preview
columns = ['intentType', 'obs_collection', 'wavelength_region', 'target_name', 'dataproduct_type', 'em_min']
obs_table[:10][columns].show_in_notebook()

## Create Plotting Variables
Before we start parsing our observations table, let's recall what we want to do with it.

First, we want to plot a historical observation coverage plot, where the horizontal axis will be time and the vertical axis will be observed wavelength. We are also going to want to label or color each observation according to what mission it corresponds to. So, we are going need variables for:
* array of times of all observations = `times`
* array of wavelengths of all observations = `waves`
* array of mission names of all observations = `mission` <br> 

We will want to modify the queried data for easy visualization:
1) We will want to convert the Modified Julian Date (MJD) to a calendar year so when we plot the timeline, it will be easy to tell when each observation was made. 

2) MAST only holds columns for the minimum and maximum wavelength of the observation, but for the historic observation coverage plot, we only want one wavelength per observation. We will calculate the average of the min/max values, and use this number instead.

In [ ]:
# Parse the observations table to get the Time data
obs_times = obs_table["t_min"]

# Convert MJD to Calendar Date:
# Initialize list for times as calendar dates
times = []
# Loop through times queried from MAST
for t in obs_times:
    # Convert MJD to Julian date
    t = t + 2400000.5
    # Convert Julian date to Calendar date 
    time = pd.to_datetime(t, unit = 'D', origin = 'julian')
    # Add converted date to times list
    times.append(time.to_numpy())

# Change list to numpy array for easy plotting
times = np.array(times)

# Parse observations table to get the wavelength data
wavelength_min = obs_table["em_min"]
wavelength_max = obs_table["em_max"]
# Some older missions have wavelengths that are off by a factor of 10^9
waves = []
for wave in wavelength_min:
    if wave/1e9 >= 1:
        waves.append(wave/1e9)
    else:
        waves.append(wave)
#change list to numpy array
waves = np.array(waves)

#Parse the observations table to get the mission names data
mission = obs_table["obs_collection"]
mission = np.array(mission)

## Plotting Historical Observation Coverage

We want to visualize the history of observations of **3c273** according to the wavelength of the observation. This plot will tell us much more if we change the color of each point to indicate which mission made the observation. 

In [ ]:
#make plot axes
fig = plt.figure()
fig.set_size_inches(15,10)
ax = fig.add_subplot()
        
#get color for every unique mission name
cm = plt.cm.get_cmap("plasma") 
num_colors = len(np.unique(mission))
ax.set_prop_cycle(color = [cm(1.*i/num_colors) for i in range(num_colors)])
marker = itertools.cycle(('^', 'o', 's','*')) 

#loop through the mission names
for i in np.unique(mission):
    #filter times and wavelengths by mission name
    ind = np.where(mission == i)
    #plot it
    ax.scatter(times[ind], waves[ind], label = i, s = 100, marker = next(marker))#, edgecolor = ecolor)

# plt.plot(times, wavelengths)

#place the legend
plt.legend()  
#set the limits of the y-axis
plt.ylim(10**-9,10**3)
# set the label of the x and y axes
plt.xlabel("Time of Observation [Year]", fontsize = 15)
plt.ylabel("Wavelength of Observation [nm]", fontsize = 15)
#show the plot
plt.show()

# Plot a Spectrum

Now, we want to plot a spectrum from one of our observations, where the x-axis will be wavelength and the y-axis will be flux (brightness).

We can use our historical observational coverage plot to choose which observation to plot. Let's pick one from the Hubble Space Telescope (HST). Its high resolution coverage of the ultraviolet makes emission and absorption lines in this region clear; this can help us deduce the composition of **3c273**.

### Query HST for Spectra

In [ ]:
# Query all the observations of 3c273 from the Hubble Space Telescope
hst_table = Observations.query_criteria(objectname="3c273",radius="10 arcsec", 
                                        dataproduct_type="spectrum", obs_collection="HST")

# Let's print out some relevant columns of this table
columns = ["instrument_name","filters","target_name","obs_id","calib_level","t_exptime"]
hst_table[columns].show_in_notebook(display_length=10)

### List Available Instrument and Filter Combinations

Most telescopes will have multiple instruments and observing modes. Here we'll print a summary of the filters and instruments that are available for our search results. This is useful to us because the different instruments aboard Hubble will cover different wavelength ranges.

In our table, we'll also create two new columns: average exposure time and maximum exposure time. This can help constrain searches for faint objects, or targets that need a longer exposure to be fully resolved.

In [ ]:
hst_table['count'] = 1
columns = hst_table.group_by(["instrument_name","filters"])
summary_table = columns["instrument_name","filters","count"].groups.aggregate(np.sum)

# Create two new columns: the average exposure time, and the maximum
summary_table["avg_exptime"] = columns['t_exptime'].groups.aggregate(np.mean)
summary_table["max_exptime"] = columns['t_exptime'].groups.aggregate(np.max)
summary_table["avg_exptime"].format = ".1f"
summary_table["max_exptime"].format = ".1f"

#Take a look at the summary table
summary_table

### Select Observations with a Specific Instrument and Filter Combination

We are interested in an ultraviolet observation that has an appropriate number of observations. Many of these instrument and filter combinations only have 1 or 2 observations. The COS G130M data looks like a good possibility. Let's look at the observations for that mode. 

In [ ]:
g130m_table = hst_table['obsid','obs_id','target_name','calib_level',
                        't_exptime','filters','em_min','em_max'][hst_table['filters']=='G130M']

# Print out the table of data for this specific filter configuration
g130m_table

In [ ]:
# We'll take the longest exposure in this filter and plot the spectrum
sel_table = g130m_table[np.argmax(g130m_table['t_exptime'])]

In [ ]:
#Take a look at the selected observation's data table
sel_table

### Filtering for Relevant Products
Our table of selected observations includes not just the spectra but also RAW files, dark scans, bias scans, and others, all of which are used in the calibration of the data. These are not necessary for plotting the spectrum, so we'll filter them out by selecting only the Minimum Recommended Products which are marked as intended for science.

In [ ]:
# Query the observations from MAST to get a list of products for our selected observation
data_products = Observations.get_product_list(sel_table)

# Get the minimum required products
filtered = Observations.filter_products(data_products, productType='SCIENCE', 
                                        productGroupDescription='Minimum Recommended Products')

# Let's take a look at the products available for our selected observation
filtered

### Download the Data to Plot

In [ ]:
# Download the data for our selected observation
data = Observations.download_products(filtered)

We've downloaded a [Flexible Image Transport System](https://fits.gsfc.nasa.gov/) (FITS) file. This is a very common file type used in astronomy for holding data of multiple dimensions. FITS files can hold images but can also contain spectral and temporal information. 

We can read the columns of our FITS file to see that it holds two segements of data for this observation, FUVA and FUVB. These are two different subsections of the far-ultraviolet spectrum that Hubble observes.

To plot a spectrum, we'll need to get the data from the `Wavelength` and `Flux` columns. 

In [ ]:
#Take a peek at the FITS file we downloaded
filename = data['Local Path'][0]
fits.info(filename)

#Read the table with the spectrum from the FITS file
tab = Table.read(filename)
tab

### Plot the Spectrum

In [ ]:
#Make the figure and set the font size globally
plt.rcParams.update({"font.size": 14})
plt.figure(1,(15,8))

#Gather the arrays from our data table
waves = tab['WAVELENGTH']
fluxes = tab["FLUX"]
segment = tab['SEGMENT']

#You'll notice from our data table that there are two segments to this observation, FUV A and FUV B
#Let's parse the spectra by their segment and plot them separately
ind_A = np.squeeze(np.where(fluxes != 0) and np.where(segment == 'FUVA'))
waves_A = waves[ind_A]
fluxes_A = fluxes[ind_A]
ind_B = np.squeeze(np.where(fluxes != 0) and np.where(segment == 'FUVB'))
waves_B = waves[ind_B]
fluxes_B = fluxes[ind_B]

#Plot both segments
plt.plot(waves_B, fluxes_B, label = "FUV B", color = 'lightcoral')
plt.plot(waves_A, fluxes_A, label = "FUV A", color = 'turquoise')

#Set the x and y axes labels and the title
plt.xlabel('Wavelength [{}]'.format(tab['WAVELENGTH'].unit))
plt.ylabel('Flux [{}]'.format(tab['FLUX'].unit))
plt.title("HST Spectrum of 3c273")

#plot the legend
plt.legend()

#give the figure a tight layout (optional)
plt.tight_layout()

# Exercises
### Recognizing Familiar Emission Lines
Look at the spectrum we plotted, does anything stand out to you? 

In astronomy, spectral features at specific wavelengths are indicative of known elements. For example, an emission line at 1216 Angstroms is called Lyman Alpha. It is produced when an orbital electron of a hydrogen atom drops from the first excited state down to the ground state, emitting a photon. 

Does **3c273** have a Lyman Alpha emission line? Plot a vertical line at 1216 Angstroms to find out.

# Citations

* [Citation for `astropy`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)


# About this Notebook

**Author**: Emma Lieb <br>
**Last updated:** Sep 2022

If you have questions, comments, or other feedback, please contact the Archive HelpDesk at archive@stsci.edu.

***
 <img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

[Return to top of page](#Historical-Quasar-Observations)